# Exercise 2: Temporal Hello World

**Goal:** Create a basic Temporal workflow with activities to understand durability and retries.

**Timebox:** 15 minutes

## What You'll Learn

- How to define Temporal workflows and activities
- How to run a workflow and observe execution in the Temporal UI
- How activities provide automatic retries and durability


> ** Tip:** If you get stuck, check the solution notebook in the `solutions/` directory! Each exercise has a corresponding complete solution that you can reference.
### Architecture Pattern 

```
Workflow Execution Request 
    ↓
Temporal Workflow (orchestration) 
    ↓
Temporal Activity (unit of work) 
    ↓
[Process data, call APIs, etc.]
    ↓
Activity completes [OK]
    ↓
Workflow returns result 
    ↓
Return to caller [OK]
```

**Key Benefits:**
- [OK] Activities automatically retry on failure
- [OK] Workflow state persists across crashes
- [OK] Full execution history in Temporal UI

## Prerequisites

Before starting, ensure you have:

1. **Temporal Server Running:** Start it from the root directory:
   - Open `temporal_installation.ipynb` and run all cells to download and start Temporal (if you haven't already)

## Setup

Import required libraries and verify Temporal server.

In [ ]:
%pip install --quiet temporalio nest-asyncio

import asyncio
import nest_asyncio
from datetime import timedelta, datetime
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker, UnsandboxedWorkflowRunner

print("[OK] Imports loaded")

## Step 1: Define an Activity

Activities are functions that do the actual work in Temporal. They can:
- Call external APIs
- Access databases
- Automatically retry on failure

**TODO:** Complete the `process_data` activity function.

**Hints:**
- Use `activity.logger.info()` to log activity lifecycle events
- Use `await asyncio.sleep(1)` to simulate work
- Transform the input data (e.g., uppercase it)
- Return the processed result as a string



In [ ]:
@activity.defn
async def process_data(data: str) -> str:
    """Activity that simulates work by upper-casing the payload."""
    # TODO: Log that the activity started using activity.logger.info()
    # Hint: activity.logger.info(" Activity started")
    
    # TODO: Simulate work with a 1-second sleep
    # Hint: await asyncio.sleep(1)
    
    # TODO: Transform the data (uppercase it) and create a result string
    # Hint: result = f"Processed: {data.upper()}"
    
    # TODO: Log that the activity completed
    # Hint: activity.logger.info("[OK] Activity completed")
    
    # TODO: Return the result
    # Hint: return result
    
    raise NotImplementedError("Complete the TODOs above")


## Step 2: Define a Workflow

Workflows orchestrate activities. They define the business logic and coordination.

**TODO:** Complete the `HelloWorkflowTemporal` class.

**Hints:**
- Use `workflow.logger.info()` to log workflow lifecycle events
- Call the activity with `await workflow.execute_activity()`
- Pass the activity function, args as a list, and a timeout
- Return a descriptive string with the activity result


In [ ]:
@workflow.defn
class HelloWorkflowTemporal:
    """Workflow that orchestrates the activity call."""

    @workflow.run
    async def run(self, name: str) -> str:
        # TODO: Log that the workflow started
        # Hint: workflow.logger.info(" Workflow started")
        
        # TODO: Execute the process_data activity
        # Hint: result = await workflow.execute_activity(
        #     process_data,
        #     args=[f"Hello {name}"],
        #     start_to_close_timeout=timedelta(seconds=10),
        # )
        
        # TODO: Log that the workflow finished
        # Hint: workflow.logger.info("[OK] Workflow finished")
        
        # TODO: Return the workflow result
        # Hint: return f"Workflow result: {result}"
        
        raise NotImplementedError("Complete the TODOs above")

## Step 3: Run the Workflow

Execute the workflow with a worker that processes both workflows and activities.

**What this does:**
1. Connects to the Temporal server at `localhost:7233`
2. Creates a unique workflow ID with timestamp
3. Starts a worker with our workflow and activity
4. Executes the workflow and waits for the result
5. Displays the result and a link to the Temporal UI

**No TODOs here** - this cell is complete and ready to run once you finish the TODOs above!



In [ ]:
# Execute the workflow
async def run_exercise() -> None:
    print("\n Exercise 2: Temporal Hello World\n")
    
    # Connect to Temporal server at localhost:7233
    client = await Client.connect("localhost:7233")
    task_queue = "hello-world-queue"
    
    # Get current time in EST and format workflow ID
    est = pytz.timezone('US/Eastern')
    now = datetime.now(est)
    workflow_id = f"02-workflow-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"
    
    print(f"Workflow ID: {workflow_id}\n")

    # Start worker with workflows and activities
    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[HelloWorkflowTemporal],  # Sets the workflow type name in UI
        activities=[process_data],
        workflow_runner=UnsandboxedWorkflowRunner(),  # Required for Jupyter notebooks
    ):
        # Execute the workflow and wait for result
        result = await client.execute_workflow(
            HelloWorkflowTemporal.run,
            "Temporal",
            id=workflow_id,
            task_queue=task_queue,
        )

    print(f"\n[OK] Workflow Result: {result}\n")
    print(
        "View execution history: "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )

# Run the workflow in a notebook-friendly way
try:
    # Check if we're already in an async context (like Jupyter notebook)
    asyncio.get_running_loop()
except RuntimeError:
    # No event loop exists, so create a new one and run our function
    # This happens in regular Python scripts or non-async environments
    asyncio.run(run_exercise())
else:
    # We're in an existing event loop (Jupyter), so we need special handling
    # Allow nested async calls in the existing event loop
    nest_asyncio.apply()
    # Run our async function in the existing loop
    await run_exercise()

## Expected Output

When you complete the TODOs, you should see output similar to:

```
 Exercise 2: Temporal Hello World

Workflow ID: 02-workflow-thu-nov-07-0230pm-est

[OK] Workflow Result: Workflow result: Processed: HELLO TEMPORAL

View execution history: http://localhost:8233/namespaces/default/workflows/02-workflow-thu-nov-07-0230pm-est
```

**What's happening:**
1. The workflow starts and logs " Workflow started"
2. The workflow executes the `process_data` activity
3. The activity logs " Activity started", processes the data, and logs "[OK] Activity completed"
4. The activity returns the processed result to the workflow
5. The workflow logs "[OK] Workflow finished" and returns the final result

**Check the Temporal UI:**
- Click the link to view the execution history
- See the workflow events, activity execution, and logs
- This visibility is one of Temporal's key benefits!



## Troubleshooting

### Error: `Failed to connect to Temporal server`

**Fix:** Ensure Temporal server is running:
1. Open `temporal_installation.ipynb` in the project root
2. Run all cells to install Temporal CLI and start the dev server
3. **Verify:** In Codespaces, go to the **Ports** tab → Find port **8233** → Click the **Globe icon**  to view the Temporal UI

### Error: `NotImplementedError`

**This means you haven't completed the TODOs yet!**
- Go back to the activity and workflow cells
- Follow the hints to implement the missing code
- Check the solution notebook if you get stuck

### Workflow completes but shows no logs

**Possible causes:**
- Make sure you're using `activity.logger.info()` and `workflow.logger.info()`
- Check the Temporal UI for logs - they may only appear there
- Ensure the worker is running (it starts automatically in the `async with Worker` block)

## Next Steps

Once you've completed this exercise, you'll understand:

- [OK] How to define Temporal activities (units of work)
- [OK] How to define Temporal workflows (orchestration logic)
- [OK] How to run workflows with workers
- [OK] How to view execution history in the Temporal UI

**Ready for more?** Proceed to:

- **[Exercise 3: Durable Agent](../03_durable_agent/exercise.ipynb)** - Combine AI agents with Temporal for production durability!
- **[Exercise 4: Agent Routing](../04_agent_routing/README.md)** - Build a multi-agent routing system